In [2]:
import pandas as pd
import src
import nltk

In [3]:
df = pd.read_pickle('./data/db_master_clean.pickle')

In [4]:
df.shape

(20551, 10)

In [5]:
df[:1]

,urn,name,url,ofsted,rawtext,scraping,clean_text,tokens,joined_tokens,agressive_clean
0,100322,Avonmore Primary School,http://www.avonmore.lbhf.sch.uk/,2.0,NOLINKABOUT,-1,,,,


In [6]:
df[df.agressive_clean==''].shape

(14110, 10)

In [7]:
df.keys()

Index([u'urn', u'name', u'url', u'ofsted', u'rawtext', u'scraping',
       u'clean_text', u'tokens', u'joined_tokens', u'agressive_clean'],
      dtype='object')

In [8]:
df[df.agressive_clean!=''].ofsted.value_counts()

2.0    3500
1.0    1500
3.0    1243
4.0     198
Name: ofsted, dtype: int64

In [23]:
for text in df.loc[df.agressive_clean!=''].loc[df.ofsted==4.0].agressive_clean:
    print text
    print 

['WIX houses three streams: English, Bilingual and French. Our school is a wonderful, close, and on-site cooperation between WIX Primary School and the cole de WIX (Lyce Franais Charles de Gaulle). We value and cherish the rich mixture of cultures and world languages that are represented at WIX.', "Based in vibrant Battersea in the borough of Wandsworth and right on the doorstep of the expansive sports areas of Clapham Common, WIX School sits just three miles from the heart of London, with all its world-class educational attractions. WIX classes make regular trips to among others: the National Gallery, the Natural History Museum, Tate Modern, the Science Museum, London Zoo, Kew Gardens and the British Museum. However, it's WIX's community spirit and its location in a quiet leafy residential street which gives it a village feel. Children have the opportunity to join community groups such as Beavers, Scouts and Guides, while local karate, judo, rugby, football, tennis, ballet, music and 

In [45]:
for token in df.loc[df.agressive_clean!=''].loc[df.ofsted==4.0].tokens:
    print token
    print 

[u'wix', u'house', u'stream', u'english', u'bilingual', u'french', u'school', u'wonderful', u'close', u'site', u'cooperation', u'wix', u'primary', u'school', u'cole', u'wix', u'lyce', u'franais', u'charles', u'gaulle', u'value', u'cherish', u'rich', u'mixture', u'culture', u'world', u'language', u'represent', u'wix', u'base', u'vibrant', u'battersea', u'borough', u'wandsworth', u'right', u'doorstep', u'expansive', u'sport', u'area', u'clapham', u'common', u'wix', u'school', u'sit', u'mile', u'heart', u'london', u'world', u'class', u'educational', u'attraction', u'wix', u'class', u'make', u'regular', u'trip', u'national', u'gallery', u'natural', u'history', u'museum', u'tate', u'modern', u'science', u'museum', u'london', u'zoo', u'kew', u'gardens', u'british', u'museum', u'wix', u'community', u'spirit', u'location', u'quiet', u'leafy', u'residential', u'street', u'village', u'feel', u'child', u'opportunity', u'join', u'community', u'group', u'beavers', u'scouts', u'guides', u'local', u'

In [9]:
df_filtered = df[(df.agressive_clean!='')]

In [10]:
df_filtered.shape

(6441, 10)

In [11]:
text = df_filtered.agressive_clean; y = df_filtered.ofsted

In [12]:
y.value_counts()

2.0    3500
1.0    1500
3.0    1243
4.0     198
Name: ofsted, dtype: int64

In [14]:
text.iloc[0]

['Provide a safe and supportive environment in which children can learn.',
 'Provide a place where all children, staff and the wider community feel welcome, respected and valued.',
 'Promote opportunities for all children and celebrate their achievement.',
 'Encourage self-expression, creativity and risk taking.',
 'Are inclusive and do all we can to overcome barriers to learning.',
 'Have happy children who enjoy coming to school.',
 'Please see our british-values.pdf  document to see some of the ways we do this.']

##### Downloading NLTK and spcaky data

In [15]:
nltk.download(download_dir='/Users/cgg/nltk_data')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

https://github.com/NSchrading/intro-spacy-nlp/blob/master/Intro_spaCy_NLP.ipynb

In [41]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.en import English
import string

def make_tokens(text_series, len_thr=5):
    """ """
    parser = English()
    tokens_list = []
    for i in range(text_series.shape[0]):
        text = text_series.iloc[i]

        # Get the tokens using spaCy
        text = unicode(text)
        tokens = parser(text)
        
#         return tokens
#         print tokens 
        
        # Stemming/Lemmatizing
        lemmas = []
        for tok in tokens:
            lemmas.append(tok.lemma_.lower().strip()
                          if tok.lemma_ != "-PRON-"
                          else tok.lower_)
        tokens = lemmas

#         print tokens

        # Remove stopwords
        STOPLIST = unicode(stopwords.words('english') +
                           ["n't", "'s", "'m", "ca"] +
                           list(ENGLISH_STOP_WORDS))
#         print STOPLIST
        tokens = [tok for tok in tokens if tok not in STOPLIST]

        # Remove some punctuation
        SYMBOLS = unicode(["-----", "---", "...", "â", "â", "'ve"])
        tokens = [tok for tok in tokens if tok not in SYMBOLS]

        # Remove punctuation and some strange things
        SYMBOLS = unicode(" ".join(string.punctuation).split(" "))
        for sym in SYMBOLS:
            tokens = [tok.replace(sym, '') for tok in tokens]

        # Remove whitespace
        while "" in tokens:
            tokens.remove("")
        while " " in tokens:
            tokens.remove(" ")
        while "\n" in tokens:
            tokens.remove("\n")
        while "\n\n" in tokens:
            tokens.remove("\n\n")

        # Remove tokens with less than len_thr words
        if len(tokens) < len_thr:
            tokens = ''
        
        tokens_list.append(tokens)
    return tokens_list


In [42]:
%%time 
tokens = make_tokens(text[:3], len_thr=5)

CPU times: user 9.2 s, sys: 2.36 s, total: 11.6 s
Wall time: 12.4 s


In [19]:
len(tokens)

3

In [85]:
text.iloc[2]

['Bus Routes The  bus goes along Dalling Road, and many other buses stop in King Street nearby. Ravenscourt Park Station, on the District Line, is very close to the School.',
 'Flora Gardens is a mixed community primary school of Hammersmith & Fulham Local Authority.',
 'Flora Gardens Primary School is popular and very well thought of locally.  It enjoys a good reputation amongst parents in the surrounding area for its friendliness, high standards, care for pupils and pleasant environment.',
 'The school is located in a single-storey building which opened in . The surrounding environment is very pleasant, with extensive grassy areas in the playgrounds. There is a wild garden with a pond, vegetable plots for each class, and many trees including a small, recently planted orchard.',
 "The school is very well equipped and resourced, with good learning resources for all curriculum areas.  The school has networked classroom PCs, laptops and iPad's, equipped with an excellent range of up-to-d

In [86]:
tokens[2]

[u'bus',
 u'routes',
 u'bus',
 u'dalling',
 u'road',
 u'bus',
 u'stop',
 u'king',
 u'street',
 u'nearby',
 u'ravenscourt',
 u'park',
 u'station',
 u'district',
 u'line',
 u'close',
 u'school',
 u'flora',
 u'gardens',
 u'mixed',
 u'community',
 u'primary',
 u'school',
 u'hammersmith',
 u'fulham',
 u'local',
 u'authority',
 u'flora',
 u'gardens',
 u'primary',
 u'school',
 u'popular',
 u'think',
 u'locally',
 u'enjoy',
 u'good',
 u'reputation',
 u'parent',
 u'surround',
 u'area',
 u'friendliness',
 u'high',
 u'standard',
 u'care',
 u'pupil',
 u'pleasant',
 u'environment',
 u'school',
 u'locate',
 u'single',
 u'storey',
 u'building',
 u'open',
 u'surround',
 u'environment',
 u'pleasant',
 u'extensive',
 u'grassy',
 u'area',
 u'playground',
 u'wild',
 u'garden',
 u'pond',
 u'vegetable',
 u'plot',
 u'class',
 u'tree',
 u'include',
 u'small',
 u'recently',
 u'plant',
 u'orchard',
 u'school',
 u'equipped',
 u'resourced',
 u'good',
 u'learning',
 u'resource',
 u'curriculum',
 u'area',
 u'school

##### Trying ngrams

In [21]:
from nltk.util import ngrams

In [87]:
bigrams = ngrams(tokens[2], n=2)

In [88]:
for i in bigrams: print i

(u'bus', u'routes')
(u'routes', u'bus')
(u'bus', u'dalling')
(u'dalling', u'road')
(u'road', u'bus')
(u'bus', u'stop')
(u'stop', u'king')
(u'king', u'street')
(u'street', u'nearby')
(u'nearby', u'ravenscourt')
(u'ravenscourt', u'park')
(u'park', u'station')
(u'station', u'district')
(u'district', u'line')
(u'line', u'close')
(u'close', u'school')
(u'school', u'flora')
(u'flora', u'gardens')
(u'gardens', u'mixed')
(u'mixed', u'community')
(u'community', u'primary')
(u'primary', u'school')
(u'school', u'hammersmith')
(u'hammersmith', u'fulham')
(u'fulham', u'local')
(u'local', u'authority')
(u'authority', u'flora')
(u'flora', u'gardens')
(u'gardens', u'primary')
(u'primary', u'school')
(u'school', u'popular')
(u'popular', u'think')
(u'think', u'locally')
(u'locally', u'enjoy')
(u'enjoy', u'good')
(u'good', u'reputation')
(u'reputation', u'parent')
(u'parent', u'surround')
(u'surround', u'area')
(u'area', u'friendliness')
(u'friendliness', u'high')
(u'high', u'standard')
(u'standard', u'c

In [30]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

In [89]:
finder = BigramCollocationFinder.from_words(tokens[2])

In [90]:
finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)

[(u'flora', u'gardens'),
 (u'fundraising', u'event'),
 (u'gender', u'ability'),
 (u'gifted', u'talented'),
 (u'child', u'flourish'),
 (u'broad', u'vision'),
 (u'develop', u'broad'),
 (u'flourish', u'reach'),
 (u'high', u'standard'),
 (u'irrespective', u'race')]

In [93]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
value = finder.score_ngrams(bigram_measures.likelihood_ratio)
for bigram, score in value[:3]:
    print bigram, score

(u'flora', u'gardens') 59.1338766022
(u'fundraising', u'event') 27.3275284758
(u'gender', u'ability') 27.3275284758


It does the same as previously. Do the bigrams appear in the original text? Not really!

In [92]:
text.iloc[2]

['Bus Routes The  bus goes along Dalling Road, and many other buses stop in King Street nearby. Ravenscourt Park Station, on the District Line, is very close to the School.',
 'Flora Gardens is a mixed community primary school of Hammersmith & Fulham Local Authority.',
 'Flora Gardens Primary School is popular and very well thought of locally.  It enjoys a good reputation amongst parents in the surrounding area for its friendliness, high standards, care for pupils and pleasant environment.',
 'The school is located in a single-storey building which opened in . The surrounding environment is very pleasant, with extensive grassy areas in the playgrounds. There is a wild garden with a pond, vegetable plots for each class, and many trees including a small, recently planted orchard.',
 "The school is very well equipped and resourced, with good learning resources for all curriculum areas.  The school has networked classroom PCs, laptops and iPad's, equipped with an excellent range of up-to-d

In [10]:
src.resample_classes?